In [1]:
#Choose CMIP Models bsed on ENSO diversity

In [2]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index

In [3]:
import xarray
import numpy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from eofs.xarray import Eof

In [4]:
import warnings
warnings.filterwarnings('ignore')

For all models, calculate the alpha, and plot eof 1 and 2. The alpha is based on monthly eof1/2 for Dec-Jan-Feb only



In [5]:
#modelSet=_model.scenarioMip[[4,6,7,29,30,31,32,33,34,36,37],:]

In [6]:
modelSet=_model.scenarioMip

In [7]:
modelSet

array([['CSIRO-ARCCSS', 'ACCESS-CM2', 'r1i1p1f1', 'r1i1p1f1'],
       ['CSIRO', 'ACCESS-ESM1-5', 'r1i1p1f1', 'r1i1p1f1'],
       ['AWI', 'AWI-CM-1-1-MR', 'r1i1p1f1', 'r1i1p1f1'],
       ['BCC', 'BCC-CSM2-MR', 'r1i1p1f1', 'r1i1p1f1'],
       ['CAMS', 'CAMS-CSM1-0', 'r1i1p1f1', 'r1i1p1f1'],
       ['CAS', 'CAS-ESM2-0', 'r1i1p1f1', 'r1i1p1f1'],
       ['NCAR', 'CESM2', 'r1i1p1f1', 'r10i1p1f1'],
       ['NCAR', 'CESM2-WACCM', 'r1i1p1f1', 'r1i1p1f1'],
       ['THU', 'CIESM', 'r1i1p1f1', 'r1i1p1f1'],
       ['CMCC', 'CMCC-CM2-SR5', 'r1i1p1f1', 'r1i1p1f1'],
       ['CMCC', 'CMCC-ESM2', 'r1i1p1f1', 'r1i1p1f1'],
       ['CNRM-CERFACS', 'CNRM-CM6-1', 'r1i1p1f2', 'r1i1p1f2'],
       ['CNRM-CERFACS', 'CNRM-ESM2-1', 'r1i1p1f2', 'r1i1p1f2'],
       ['CCCma', 'CanESM5', 'r1i1p1f1', 'r1i1p1f1'],
       ['CCCma', 'CanESM5-CanOE', 'r1i1p2f1', 'r1i1p2f1'],
       ['EC-Earth-Consortium', 'EC-Earth3', 'r1i1p1f1', 'r1i1p1f1'],
       ['EC-Earth-Consortium', 'EC-Earth3-CC', 'r1i1p1f1', 'r1i1p1f1'],
       ['

In [ ]:
%run utils/ecIndex.ipynb

eofList = list()

for iModel in modelSet:
    try:
        print(iModel[1])
        
        tsXr = xarray.concat([fh.loadModelData(iModel[1], 'tos_Omon', 'historical', iModel[3], 
                                               chunks={'time':120}).tos, 
                              fh.loadModelData(iModel[1], 'tos_Omon', 'ssp585', iModel[3], 
                                               chunks={'time':120}).tos], 
                             dim='time')

        djfTsXr=tp.averageForTimePeriod(
            tsXr.to_dataset(name='enso')).rename({'year':'time'})

        indeces, pFit, eofsXr = ecIndex(tsXr)
        
        eofList.append(indeces)
        
        fig=plt.figure(figsize=(18,12))

        for mode in [0,1]:
            ax=fig.add_subplot(1,2,mode+1,
                           projection=ccrs.Miller(central_longitude=210))

            ensoPlotter(eofsXr.sel(mode=mode), ax)

            plt.title("mode " + str(mode))

        plt.show()

        #indecesDjf, pFit, eofsXr = ecIndex(djfTsXr)
        
        print(str(indeces.alpha.values)) #+ ' ... ' + str(indecesDjf.alpha.values))
        alphas.append(indeces.alpha.values)
        fig = plt.figure()

        plt.scatter(indeces.pc1, indeces.pc2)

        plt.plot(*pFit.linspace(), 'r')
        
        plt.show()
        
    except Exception as e:
        print(e)

ACCESS-CM2
/g/data/oi10/replicas/CMIP6/CMIP/
tos_Omon_.*?ACCESS-CM2_historical_r1i1p1f1_.*?_.*
Files (filter term: tos_Omon_.*?ACCESS-CM2_historical_r1i1p1f1_.*?_.* ) not found, possibly test name is wrong
ACCESS-ESM1-5
/g/data/oi10/replicas/CMIP6/CMIP/
tos_Omon_.*?ACCESS-ESM1-5_historical_r1i1p1f1_.*?_.*
Files (filter term: tos_Omon_.*?ACCESS-ESM1-5_historical_r1i1p1f1_.*?_.* ) not found, possibly test name is wrong
AWI-CM-1-1-MR
/g/data/oi10/replicas/CMIP6/CMIP/
tos_Omon_.*?AWI-CM-1-1-MR_historical_r1i1p1f1_.*?_.*
['/g/data/oi10/replicas/CMIP6/CMIP//AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Omon/tos/gn/v20181218/tos_Omon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_188101-189012.nc', '/g/data/oi10/replicas/CMIP6/CMIP//AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Omon/tos/gn/v20181218/tos_Omon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_186101-187012.nc', '/g/data/oi10/replicas/CMIP6/CMIP//AWI/AWI-CM-1-1-MR/historical/r1i1p1f1/Omon/tos/gn/v20181218/tos_Omon_AWI-CM-1-1-MR_historical_r1i1p1f1_gn_195101-196012.nc

Models can be chosen on some criteria

alpha < -0.15 looks appropriate from Cai et al 2018, however as the methodology doesn't match exactly, this is probably not right and should be calculated from observations.

In [9]:
ecXr=xarray.concat(eofList, 'model')

ValueError: must supply at least one object to concatenate

In [ ]:
ecXr.model.values

In [ ]:
for i in range(0,30):
    ds=ecXr.isel(model=i)
    print(str(ds.model.values) + ' : ' + str(ds.alpha.values))

In [ ]:
wangAnswer=modelSet[[4,9,#11,
                     12,15,17,30,29,22,31,32,33,21,6,7,35,36,#20,
                     8],:]

In [ ]:
for i in wangAnswer:
    ds=ecXr.sel(model=i[1])
    print(str(ds.model.values) + ' : ' + str(ds.alpha.values) + '   ' + i[3])